In [ ]:
# default_exp train.message

# Message

Using Protocol Buffer to setup the types of objects I need:

* subject
* treatment
* model
* evaluation
* expectation
* invocation

There could be more, but this handles the lifecycle of a data product:

* Define the data or type of data we're working with. This can come across a stream, be found in file, or resemble academic data sets.
* Learn how to learn from the data, defining a treatment that builds a model, applies a heuristic, or somehow transforms the data.
* Store a model with its meta data, so we know what it is, when it was built, how it was built, etc.
* Store evaluations of data transforms in standard ways, so they can be used in reporting and progress management.
* Convert data into data expectations. This works on input data to a model as well as output data from a model. This can define a distribution, or any rules that make the data valid or alarming.
* Create external interfaces to functions, models, and API calls. This is especially useful with all of my open keyword function signatures.

In [ ]:
#hide
pwd = %pwd
if pwd.split('/')[-1] == 'nbs':
    %cd ..

/Users/davidrichards/codes/hydra/lab


In [ ]:
#export
import google.protobuf
import lab.train.messages_pb2 as messages_pb2
import zlib

In [ ]:
#export
def serialize(message):
    """Convert a message to a binary string."""
    return message.SerializeToString()

def kind_to_class(kind, parent=messages_pb2):
    """Find a Message class by reference.
    Allows a recursive search for a message class.
    The message I used to test this no longer exists."""
    if isinstance(kind, type): return kind
    if isinstance(kind, str):
        kind, *tail = kind.split('.')
        if hasattr(parent, kind):
            if len(tail) > 0:
                parent = kind_to_class(kind, parent=parent)
                if parent:
                    return kind_to_class('.'.join(tail), parent=parent)
            return kind_to_class(getattr(parent, kind))

def deserialize(s, kind):
    """Deserialize a string as a kind of message."""
    cls = kind_to_class(kind)
    if cls is None: return False
    message = cls()
    message.ParseFromString(s)
    return message

DEFAULT_COMPRESSION = zlib.Z_BEST_COMPRESSION
COMPRESSION_LEVELS = dict(
    best_speed=zlib.Z_BEST_SPEED,
    best_compression=zlib.Z_BEST_COMPRESSION,
    default=zlib.Z_DEFAULT_COMPRESSION,
)

def compress(message, level=DEFAULT_COMPRESSION):
    """Compress a message."""
    if isinstance(level, str): level = COMPRESSION_LEVELS.get(level, DEFAULT_COMPRESSION)
    s = serialize(message)
    return zlib.compress(s, level)

def decompress(compressed, kind):
    """Decompress a compressed message, given the kind of message."""
    cls = kind_to_class(kind)
    if cls is None: return False
    decompressed = zlib.decompress(compressed)
    return deserialize(decompressed, cls)

def to_dict(message):
    """Create a dictionary from a message, including all defaults."""
    fields = dict(message.DESCRIPTOR.fields_by_name)
    return {k:getattr(message, k) for k in fields.keys()}

In [ ]:
assert kind_to_class(messages_pb2.Subject) == messages_pb2.Subject
assert kind_to_class('Subject') == messages_pb2.Subject

In [ ]:
#export
class MessageBuilder:
    """Create Protocol Buffer messages from Python
    data types."""
    
    DEFAULT_MESSAGE_CLASS = messages_pb2.Model
    
    def __init__(self, **kw):
        self.kw = kw
        
    def _class_from_object(self, o):
        return kind_to_class(o)
        
    @property
    def _message_class(self):
        if hasattr(self, '__message_class'): return self.__message_class
        o = self.kw.get('kind', self.DEFAULT_MESSAGE_CLASS)
        self.__message_class = kind_to_class(o)
        return self.__message_class
    
    @property
    def instance(self):
        if hasattr(self, '_instance'): return self._instance
        self._instance = self._message_class()
        return self._instance

    @property
    def _fields(self):
        if hasattr(self, '__fields'): return self.__fields
        self.__fields = dict(self._message_class.DESCRIPTOR.fields_by_name)
        return self.__fields
            
    def build(self, data):
        # TODO: add recursion and type checking.
        data = {k:v for k, v in data.items() if k in self._fields}
        try:
            return self._message_class(**data)
        except:
            # Something is wrong...just ignore it for now...
            pass
        
    def __call__(self, **data):
        return self.build(data)
    
subject_builder = MessageBuilder(kind='Subject')
treatment_builder = MessageBuilder(kind='Treatment')
model_builder = MessageBuilder(kind='Model')
evaluation_builder = MessageBuilder(kind='Evaluation')
expectation_builder = MessageBuilder(kind='Expectation')
invocation_builder = MessageBuilder(kind='Invocation')

In [ ]:
data = dict(name='bar', evaluations=[{'name': 'accuracy', 'values': [1,2,3,4]}])
message = model_builder(**data)
serialized = serialize(message)
compressed = compress(message)
decompressed = decompress(compressed, messages_pb2.Model)
assert decompressed.name == 'bar'
assert decompressed.evaluations[0].name == 'accuracy'
assert 'version' in to_dict(message)

So, serializing, deserializing, compressing, and decompressing all work. Also, creating a dictionary from a message works, even with defaults.

In [ ]:
assert MessageBuilder()._message_class == MessageBuilder.DEFAULT_MESSAGE_CLASS
assert MessageBuilder(kind=messages_pb2.Model)._message_class == messages_pb2.Model
assert MessageBuilder(kind='Model')._message_class == messages_pb2.Model
assert MessageBuilder(kind=messages_pb2.Evaluation)._message_class == messages_pb2.Evaluation
assert MessageBuilder(kind='Evaluation')._message_class == messages_pb2.Evaluation

description = """
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Maecenas commodo sem id mi fringilla volutpat. Etiam sed massa sapien. Curabitur elementum, quam id feugiat tristique, tellus mauris pretium sem, sit amet congue elit lectus non dolor. Suspendisse imperdiet lectus at est posuere, vel suscipit arcu rhoncus. Integer ultricies laoreet erat eu laoreet. Pellentesque nec metus scelerisque, dignissim quam ut, suscipit magna. Sed nec purus ut felis eleifend volutpat.

Mauris consectetur metus a finibus pellentesque. Nunc eleifend tempus mollis. Nullam eget interdum enim. Sed nunc lorem, elementum vel pretium in, luctus eu turpis. In hac habitasse platea dictumst. Curabitur tincidunt placerat est, eu imperdiet enim cursus laoreet. Nam placerat varius lacus, at laoreet felis facilisis ut. Vestibulum vitae eros ante. Vestibulum mattis sem ut lacus posuere, at tincidunt augue imperdiet. Nulla congue erat vitae suscipit ultrices. Maecenas gravida egestas magna eget condimentum. Morbi vitae varius magna. Pellentesque ut diam magna. Etiam justo nibh, aliquet eget finibus at, finibus ac quam. Sed nec porta justo, ac aliquet sem. Donec sit amet elit lobortis, volutpat sapien eget, ullamcorper erat.

Nullam eu quam porttitor, rhoncus dolor id, rhoncus nulla. Morbi et elementum nunc. Orci varius natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Sed imperdiet nulla nunc, vel sollicitudin libero dapibus sed. Curabitur ultrices ante at gravida rhoncus. Nunc quis ante convallis, dapibus mi sit amet, efficitur ex. Ut magna est, egestas sit amet scelerisque sit amet, finibus a lectus. Sed vitae dignissim leo. Sed laoreet ante et sem convallis auctor. Morbi sodales ut magna ultrices sodales. Integer luctus blandit nunc. Sed a dapibus massa. Suspendisse rhoncus erat lacus, quis facilisis dui ornare dignissim. Orci varius natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Curabitur purus ante, vulputate at justo in, egestas egestas ante. Duis vitae est rutrum, luctus erat eget, luctus enim.

Aliquam eu neque nisl. Ut egestas, dui nec posuere sagittis, justo massa gravida velit, a pulvinar urna turpis in elit. Integer ex orci, scelerisque non felis quis, ornare aliquet quam. Fusce at metus at risus auctor interdum a eget ante. Aenean congue, risus ac rutrum gravida, leo tellus ullamcorper nisi, quis aliquet justo orci ut ex. Morbi sagittis augue ac dui posuere, quis vestibulum dui finibus. Nam elementum ex non diam sollicitudin, sit amet finibus velit convallis. Vivamus viverra, ipsum quis cursus tempor, neque erat tincidunt elit, a tempor dolor enim quis nibh. Morbi vestibulum massa sed tempus hendrerit. Quisque aliquet metus feugiat, pellentesque nisl id, rutrum quam. Nulla sit amet dapibus ipsum. Morbi ut lorem aliquam, lacinia tortor vitae, luctus mauris. Maecenas vehicula elementum justo gravida luctus. Maecenas suscipit pellentesque nisi ut rutrum. Praesent a convallis quam.

Donec at nisi eros. Vestibulum sed libero vitae nunc posuere aliquet. Aliquam erat volutpat. Sed lorem felis, sagittis et pulvinar id, posuere sed arcu. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec vitae dolor vulputate, iaculis velit fermentum, posuere ante. Aenean erat nisl, feugiat ac interdum sed, fringilla vitae purus. Curabitur pulvinar aliquam dui, eget suscipit augue volutpat id. Nullam auctor mollis mi, vel sodales erat scelerisque vel. Aliquam bibendum purus eu feugiat hendrerit. Pellentesque id ultrices ante. Nullam non mauris a nibh finibus commodo. Fusce vel purus vitae tortor vestibulum scelerisque sed eu erat. In hac habitasse platea dictumst. Ut feugiat magna nunc, eget tempor sapien commodo nec. Donec tincidunt augue at leo scelerisque euismod.
"""

data = dict(name='foo', description=description, foo=42)

assert model_builder.build(data).description == description
assert model_builder(**data).name ==data['name']

data = dict(name='bar', evaluations=[{'name': 'accuracy', 'values': [1,2,3,4]}])
model = model_builder(**data)
assert len(model.evaluations) == 1
